# Titanic - Feature Engineering

This cleans the data from the titanic dataset and generates two new datasets.  A lot of these ideas were sourced from here:
https://towardsdatascience.com/predicting-the-survival-of-titanic-passengers-30870ccc7e8

After further review, it looks like the above article ripped off a lot of the work done by Sina here: https://www.kaggle.com/sinakhorami/titanic-best-working-classifier

In [266]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [267]:
X_train = train_df.drop("Survived", axis=1)
Y_train = train_df["Survived"]
X_test  = test_df.drop("PassengerId", axis=1).copy()

In [268]:
# Combine will be used for sourcing our values only.  These values will then be used to manipulate that actual data sets.
def combine(train, test):
    combined = pd.concat([train,test])
    return combined

combined = combine(train_df, test_df)
all_data = [train_df,test_df]

In [269]:
total = combined.isnull().sum().sort_values(ascending=False)
percent_1 = combined.isnull().sum()/combined.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(10)

,Total,%
Cabin,1014,77.5
Survived,418,31.9
Age,263,20.1
Embarked,2,0.2
Fare,1,0.1
Ticket,0,0.0
Parch,0,0.0
SibSp,0,0.0
Sex,0,0.0
Name,0,0.0


In [270]:
# This will fix the Embark column from a S,C,Q to 0,1,2 and add missing values with the most common value
def fix_feaure_embark(data, combined):
    # Get the most common value from our combined set
    most_common = combined['Embarked'].mode()[0]
    
    # Fill in blanks with our most common value
    combined['Embarked'] = combined['Embarked'].fillna(most_common)
    for d in data:
        d['Embarked'] = d['Embarked'].fillna(most_common)
    
    # Build a dictionary of each of our unique values and their index
    ports = {}
    for idx, val in enumerate(combined['Embarked'].unique()):
        ports[val] = idx
    
    # For each dataset, replace the current value with the index value
    # from list of unique values using our ports dictionary above
    # for example: replace S -> 0, C -> 1, Q -> 2
    for d in data:
        d['Embarked'] = d['Embarked'].map(ports)    
    
    return data

all_data = fix_feaure_embark(all_data, combined)
all_data[0]['Embarked'].describe()

count    891.000000
mean       0.361392
std        0.635673
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        2.000000
Name: Embarked, dtype: float64

In [271]:
# This extracts the title from the name and creates a new feature
def add_feature_title(data, combined):
    # Add the title column with the actual title like 'Mr.'
    for d in data:
        d['Title'] = [n.split(',')[1].strip().split(' ')[0] for n in d['Name']]
    combined['Title'] = [n.split(',')[1].strip().split(' ')[0] for n in combined['Name']]
    
    # Build a dictionary of each of our unique values and their index
    ports = {}
    for idx, val in enumerate(combined['Title'].unique()):
        ports[val] = idx
    
    # Replace the title values with a numeric value
    for d in data:
        d['Title'] = d['Title'].map(ports) 
    
    return data, combined

all_data, combined = add_feature_title(all_data, combined)

In [272]:
# This shows some title information
#print (combined['Title'].unique())
#print(combined.loc[combined['Title'].isin(['the'])]) #Show the record where the title is 'the'

total = combined.groupby('Title').count()['Survived']
surv = combined.groupby('Title').sum()['Survived']
#print(pd.concat([surv,total], axis=1, keys=['Surived', 'Total']))

#print(combined.groupby('Title')['Age'].describe())

In [273]:
def fix_feature_age(data, combined):
    # Fill in empty ages with the mean based on the title
    means = combined.groupby('Title')['Age'].describe()['mean']
    for d in data:
        d['Age'] = d.apply(lambda x: means[x['Title']] if pd.isnull(x['Age']) else x['Age'], axis=1)
        
    # Now we'll convert the age to a category
    for d in data:
        d.loc[d['Age'] <= 15, 'Age'] = 0
        d.loc[(d['Age'] > 15) & (d['Age'] <= 18), 'Age'] = 1
        d.loc[(d['Age'] > 18) & (d['Age'] <= 22), 'Age'] = 2
        d.loc[(d['Age'] > 22) & (d['Age'] <= 27), 'Age'] = 3
        d.loc[(d['Age'] > 27) & (d['Age'] <= 33), 'Age'] = 4
        d.loc[(d['Age'] > 33) & (d['Age'] <= 40), 'Age'] = 5
        d.loc[(d['Age'] > 40) & (d['Age'] <= 66), 'Age'] = 6
        d.loc[d['Age'] > 66, 'Age'] = 1
        d['Age'] = d['Age'].astype(int)
        #print(d['Age'].value_counts())
    
    return data

all_data = fix_feature_age(all_data, combined)

In [274]:
def add_feature_relatives(data, combined):
    for d in data:
        d['Relatives'] = d['SibSp'] + d['Parch']
    combined['Relatives'] = combined['SibSp'] + combined['Parch']
    return data, combined

all_data, combined = add_feature_relatives(all_data, combined)

In [275]:
def fix_feature_fare(data, combined):
    # Fill in blank with the mean and add a new column FareRatio
    average = combined['Fare'].mean()
    for d in data:
        d['Fare'] = d['Fare'].fillna(average)
        #d['Fare'] = (d['Fare'] - d['Fare'].mean()) / d['Fare'].std()
        d['FareRatio'] = d['Fare'] / (d['Relatives'] + 1)
        #d['FareRatio'] = (d['FareRatio'] - d['FareRatio'].mean()) / d['FareRatio'].std()
        d['Fare'] = d['Fare'].astype(int)
        d['FareRatio'] = d['FareRatio'].astype(int)
    combined['Fare'] = combined['Fare'].fillna(average)
    #combined['Fare'] = (combined['Fare'] - combined['Fare'].mean()) / combined['Fare'].std()
    combined['FareRatio'] = combined['Fare'] / (combined['Relatives'] + 1)
    #combined['FareRatio'] = (combined['FareRatio'] - combined['FareRatio'].mean()) / combined['FareRatio'].std()
    combined['Fare'] = combined['Fare'].astype(int)
    combined['FareRatio'] = combined['FareRatio'].astype(int)
        
    return data, combined

all_data, combined = fix_feature_fare(all_data, combined)

In [276]:
def fix_feature_sex(data, combined):
    # Build a dictionary of each of our unique values and their index
    ports = {}
    for idx, val in enumerate(combined['Sex'].unique()):
        ports[val] = idx
    
    # For each dataset, replace the current value with the index value
    # from list of unique values using our ports dictionary above
    # for example: replace S -> 0, C -> 1, Q -> 2
    for d in data:
        d['Sex'] = d['Sex'].map(ports)
        d['Sex'] = d['Sex'].astype(int)
    return data

all_data = fix_feature_sex(all_data, combined)

In [277]:
def add_feature_deck(data, combined):
    # Get first character of each cabin and return the unique values
    combined['Cabin'] = combined['Cabin'].fillna('X')
    decks = sorted(combined['Cabin'].astype(str).str[0].unique())
    print (decks)
    
    ports = {}
    for idx, val in enumerate(decks):
        ports[val] = idx
    
    # For each dataset, replace the current value with the index value
    # from list of unique values using our ports dictionary above
    # for example: replace S -> 0, C -> 1, Q -> 2
    for d in data:
        d['Cabin'] = d['Cabin'].fillna('X')
        d['Deck'] = d['Cabin'].astype(str).str[0].map(ports)
        d['Deck'] = d['Deck'].astype(int)
    combined['Deck'] = combined['Cabin'].astype(str).str[0].map(ports)
    combined['Deck'] = combined['Deck'].astype(int)
    
    return data, combined

all_data, combined = add_feature_deck(all_data, combined)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T', 'X']


In [278]:
ticket_prefix = all_data[0]['Ticket'].astype(str).str[0:2].unique()
print(ticket_prefix)
pattern = '|'.join(ticket_prefix)
s = all_data[0]['Ticket'].str.extract('('+ pattern + ')', expand=False)
total = all_data[0].groupby(s).size()
surv = all_data[0].groupby(s).sum()['Survived']
perc = surv / total * 100
#print (new)
# surv = all_data[0].groupby(tick_prefex).sum()['Survived']
print(pd.concat([surv,total, perc], axis=1, keys=['Surived','Total', '%']))

['A/' 'PC' 'ST' '11' '37' '33' '17' '34' '23' 'PP' '35' '24' '38' '26'
 '19' 'C.' 'A.' '75' 'SC' 'S.' '14' '31' '29' 'CA' '36' '16' 'SO' 'W.'
 '32' '41' 'A4' '27' 'C ' '22' 'Fa' '25' 'LI' '28' '45' 'F.' 'W/' '21'
 'SW' '13' '92' 'P/' '72' '39' '20' '65' '12' 'WE' '69' '54' '84' '57']
        Surived  Total           %
Ticket                            
11           50     88   56.818182
12            2      4   50.000000
13            8     11   72.727273
14            3      4   75.000000
16            8     10   80.000000
17           10     13   76.923077
19           11     16   68.750000
20            1      1  100.000000
21            0      4    0.000000
22            6      9   66.666667
23           16     36   44.444444
24           16     29   55.172414
25            7     14   50.000000
26           25     60   41.666667
27            3      4   75.000000
28            2     12   16.666667
29            9     14   64.285714
31            9     37   24.324324
32            0

In [279]:
def add_feature_ageclass(data, combined):
    for d in data:
        d['AgeClass'] = d['Age'] * d['Pclass']
    combined['AgeClass'] = combined['Age'] * combined['Pclass']
    return data, combined

all_data, combined = add_feature_ageclass(all_data, combined)

In [280]:
def drop_unnecessary_features(data):
    # This drops features that contain text or are no longer needed.
    for d in data:
        d.drop(['Name'], axis=1, inplace=True)
        d.drop(['Ticket'], axis=1, inplace=True)
        d.drop(['Cabin'], axis=1, inplace=True)
    return data
all_data = drop_unnecessary_features(all_data)
print (all_data[0])

     PassengerId  Survived  Pclass  Sex  Age  SibSp  Parch  Fare  Embarked  \
0              1         0       3    0    2      1      0     7         0   
1              2         1       1    1    5      1      0    71         1   
2              3         1       3    1    3      0      0     7         0   
3              4         1       1    1    5      1      0    53         0   
4              5         0       3    0    5      0      0     8         0   
..           ...       ...     ...  ...  ...    ...    ...   ...       ...   
886          887         0       2    0    3      0      0    13         0   
887          888         1       1    1    2      0      0    30         0   
888          889         0       3    1    5      1      2    23         0   
889          890         1       1    0    3      0      0    30         1   
890          891         0       3    0    4      0      0     7         2   

     Title  Relatives  FareRatio  Deck  AgeClass  
0        0  

In [281]:
new_train_df = all_data[0]
new_test_df = all_data[1]

new_train_df = new_train_df.drop(['PassengerId'], axis=1)

# Dropping Parch after understanding importance isn't great
#new_train_df = new_train_df.drop(['Parch', 'Embarked', 'SibSp', 'Deck', 'Age', 'Pclass', 'AgeClass', 'Relatives'], axis=1)
#new_test_df = new_test_df.drop(['Parch', 'Embarked', 'SibSp', 'Deck', 'Age', 'Pclass', 'AgeClass', 'Relatives'], axis=1)

print(new_train_df)
print(new_test_df)

new_train_df.to_csv('new_train.csv', index=False)
new_test_df.to_csv('new_test.csv', index=False)
print("New files have been created.")

     Survived  Pclass  Sex  Age  SibSp  Parch  Fare  Embarked  Title  \
0           0       3    0    2      1      0     7         0      0   
1           1       1    1    5      1      0    71         1      1   
2           1       3    1    3      0      0     7         0      2   
3           1       1    1    5      1      0    53         0      1   
4           0       3    0    5      0      0     8         0      0   
..        ...     ...  ...  ...    ...    ...   ...       ...    ...   
886         0       2    0    3      0      0    13         0      5   
887         1       1    1    2      0      0    30         0      2   
888         0       3    1    5      1      2    23         0      2   
889         1       1    0    3      0      0    30         1      0   
890         0       3    0    4      0      0     7         2      0   

     Relatives  FareRatio  Deck  AgeClass  
0            1          3     8         6  
1            1         35     2         5  
2  